In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import os

import torch
import torch.utils.data
from vis_det.model_pool.centernet.opts import opts
from vis_det.model_pool.centernet.models.model import create_model, load_model, save_model
from vis_det.model_pool.centernet.models.data_parallel import DataParallel
from vis_det.model_pool.centernet.logger import Logger
from vis_det.model_pool.centernet.datasets.dataset_factory import get_dataset
from vis_det.model_pool.centernet.trains.train_factory import train_factory
import matplotlib.pyplot as plt
import numpy as np

In [3]:
%matplotlib inline

In [4]:
os.environ["CUDA_LAUNCH_BLOCKING"]="1"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [5]:
opt = opts().parse()
opt.debug = max(opt.debug, 1)
torch.backends.cudnn.benchmark = not opt.not_cuda_benchmark and not opt.test
Dataset = get_dataset(opt.dataset, opt.task)
opt = opts().update_dataset_info_and_set_heads(opt, Dataset)


Fix size testing.
training chunk_sizes: [1]
The output will be saved to  /home/paritosh/Desktop/SEM2/16824/code/vlr-project/vis_det/vis_det/model_pool/centernet/../../exp/ddd/default
heads {'hm': 3, 'dep': 1, 'rot': 8, 'dim': 3, 'wh': 2, 'reg': 2}


In [6]:
model = create_model(opt.arch, opt.heads, opt.head_conv)
optimizer = torch.optim.Adam(model.parameters(), opt.lr)
model, optimizer, start_epoch = load_model(
      model, opt.load_model, optimizer, opt.resume, opt.lr, opt.lr_step)
Trainer = train_factory[opt.task]
trainer = Trainer(opt, model, optimizer)

val_loader = torch.utils.data.DataLoader(
      Dataset(opt, 'val'), 
      batch_size=1, 
      shuffle=False,
      num_workers=1,
      pin_memory=True
  )

loaded /data/datasets/paritosh/16824/centernet/ddd_3dop.pth, epoch 70
==> initializing kitti 3dop, val data.
loading annotations into memory...
Done (t=0.17s)
creating index...
index created!
Loaded val 3769 samples


In [7]:
train_loader = torch.utils.data.DataLoader(
      Dataset(opt, 'train'), 
      batch_size=opt.batch_size, 
      shuffle=True,
      num_workers=opt.num_workers,
      pin_memory=True,
      drop_last=True
  )

==> initializing kitti 3dop, train data.
loading annotations into memory...
Done (t=0.18s)
creating index...
index created!
Loaded train 3712 samples


# Display the Images

In [ ]:
count = 0
for iter_id, batch in enumerate(val_loader):
    count += 1
    #print(batch)
    if True:
        print(count)
        img_ = cv2.imread(batch['meta']['image_path'][0])
        plt.imshow(img_[:,:,::-1])
        plt.show()
#         initial_img = batch['input']
#         trainer.infer(batch, torch.device("cuda"), 2000)
    if count >= 100:
        break
    

# Invert the selected image using id

In [ ]:
count = 0
id = 57 
for iter_id, batch in enumerate(val_loader):
    count += 1
    if count==id:
        print(count)
        initial_img = batch['input']
        trainer.infer(batch, torch.device("cuda"), 2000)
        break


In [ ]:
plt.rcParams['figure.figsize'] = [20, 10]

In [ ]:
final_img = batch['input']

In [ ]:
def display_img(img,img_path = ""):
    img = img.squeeze().detach().cpu()
    img_disp = img.permute(1,2,0)
    plt.imshow((img_disp+1)*0.5)
    plt.imsave(img_path,((img_disp+1)*0.5).detach().cpu().numpy(), dpi=1200)

In [ ]:
display_img(final_img, f"inv_img_{id}.png")

In [ ]:
import cv2
img_ = cv2.imread(batch['meta']['image_path'][0])
img_ = cv2.resize(img_, (1280, 384))

plt.imsave(f"orig_{id}.png",img_[:,:,::-1], dpi=1200)

# Display the original Image with Detections

In [ ]:
from vis_det.model_pool.centernet.detectors.detector_factory import detector_factory
Detector = detector_factory[opt.task]
detector = Detector(opt)
inp_img = batch['input'].detach().cpu()

height, width = inp_img.shape[2:]
inp_height, inp_width = opt.input_h, opt.input_w
new_height = int(height)
new_width  = int(width)
c = np.array([new_width / 2., new_height / 2.], dtype=np.float32)
s = max(height, width) * 1.0
calib = [[721.5377197265625, 0.0, 609.559326171875, 44.85728073120117],
 [0.0, 721.5377197265625, 172.85400390625, 0.2163791060447693],
 [0.0, 0.0, 1.0, 0.0027458840049803257]]
meta = {'c': c, 's': s, 
            'out_height': inp_height // opt.down_ratio, 
            'out_width': inp_width // opt.down_ratio,
        'calib':np.array(calib, dtype=np.float32),
       'img_path': f"/home/paritosh/Desktop/SEM2/16824/code/vlr-project/vis_det/orig_{id}.png"#batch['meta']['image_path'][0]
       }
img_run = {"image":initial_img, "meta":meta}

detector.run(img_run, meta)

# Display the Inverted Image with Detections

In [ ]:
from vis_det.model_pool.centernet.detectors.detector_factory import detector_factory
Detector = detector_factory[opt.task]
detector = Detector(opt)
inp_img = batch['input'].detach().cpu()

height, width = inp_img.shape[2:]
inp_height, inp_width = opt.input_h, opt.input_w
new_height = int(height)
new_width  = int(width)
c = np.array([new_width / 2., new_height / 2.], dtype=np.float32)
s = max(height, width) * 1.0
calib = [[721.5377197265625, 0.0, 609.559326171875, 44.85728073120117],
 [0.0, 721.5377197265625, 172.85400390625, 0.2163791060447693],
 [0.0, 0.0, 1.0, 0.0027458840049803257]]
meta = {'c': c, 's': s, 
            'out_height': inp_height // opt.down_ratio, 
            'out_width': inp_width // opt.down_ratio,
        'calib':np.array(calib, dtype=np.float32),
       'img_path': f"/home/paritosh/Desktop/SEM2/16824/code/vlr-project/vis_det/inv_img_{id}.png" #batch['meta']['image_path'][0]
       }
img_run = {"image":final_img, "meta":meta}

detector.run(img_run, meta)